# PHS564 — Lecture 11 (Student)
## Time-varying treatment and confounding: marginal structural models (MSMs) (MIMIC-IV Demo)

### Learning goals
- Recognize time-varying confounding affected by prior treatment.
- Construct stabilized weights over time:
- treatment weights \(W_A\)
- censoring weights \(W_C\)
- combined weights \(W=W_A\times W_C\)
- Fit a simple MSM (weighted pooled logistic/GLM) and interpret the parameter as a **marginal causal effect** under assumptions.

### Required reading
- Hernán & Robins, MSM sections (typically Chapter 12 continuation / longitudinal chapters).

**Rules for this notebook**
- Only edit cells marked **TODO**.
- Do not change the overall structure/cell order.


In [ ]:
# Colab bootstrap (run this first if you opened from a Colab badge)
# - Clones the repo into /content/PHS564 (if needed)
# - Installs requirements
# - Adds repo to sys.path

from __future__ import annotations

import os
import sys
import subprocess
from pathlib import Path


def _in_colab() -> bool:
    return "google.colab" in sys.modules


if _in_colab():
    REPO_URL = "https://github.com/vafaei-ar/PHS564.git"
    TARGET_DIR = Path("/content/PHS564")

    if not (TARGET_DIR / "requirements.txt").exists():
        print("Cloning course repo into Colab runtime...")
        subprocess.run(["git", "clone", "--depth", "1", REPO_URL, str(TARGET_DIR)], check=True)

    os.chdir(TARGET_DIR)

    print("Installing requirements...")
    subprocess.run([sys.executable, "-m", "pip", "-q", "install", "-r", "requirements.txt"], check=True)

    if str(TARGET_DIR) not in sys.path:
        sys.path.insert(0, str(TARGET_DIR))

    print("✓ Colab setup complete. Now run the rest of the notebook.")
else:
    print("Not running in Colab; skipping Colab bootstrap.")


### Setup

This notebook is designed to run **locally** or in **Google Colab**.

**If you opened from the Colab badge (recommended):**
1) Run the first code cell titled **“Colab bootstrap”** (it clones the repo + installs requirements)
2) Run the notebook top-to-bottom.

**If you are running locally:**
- Install dependencies from `requirements.txt` (see the repo `README.md`), then run top-to-bottom.


In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reproducibility
RNG = np.random.default_rng(564)

# Locate repo root (works when running from lectures/Lxx.../student or /instructor)
THIS_DIR = Path.cwd()
REPO_ROOT = THIS_DIR
for _ in range(4):
    if (REPO_ROOT / "requirements.txt").exists() or (REPO_ROOT / "README.md").exists():
        break
    REPO_ROOT = REPO_ROOT.parent

DATA_DIR = REPO_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROC_DIR = DATA_DIR / "processed"

print("Working directory:", THIS_DIR)
print("Repo root:", REPO_ROOT)
print("Processed data dir exists:", PROC_DIR.exists())


### Build the processed cohort extract (required)

This lecture expects an analysis-ready cohort file in `data/processed/`.

If it’s missing, run the next cell to:
1) download MIMIC-IV Demo into `data/raw/` (if needed)
2) build the processed cohort extracts into `data/processed/`

**Exposure mode:** the cohort builder uses a simple teaching definition of time-varying treatment derived from vitals. You may change thresholds in the code cell.

In [ ]:
# Build processed cohort extracts from raw MIMIC-IV Demo (safe to re-run)
# This will create cohort files under data/processed/.

HR_THRESHOLD = 100.0  # defines A_t = 1 if mean daily HR > threshold
T_MAX = 7            # number of ICU days to include (teaching simplification)

try:
    from data.download_data import download_mimic_demo
    from data.build_processed_extracts_demo import build_processed_extracts

    mimic_dir = RAW_DIR / "mimic-iv-demo-2.2"
    if not mimic_dir.exists():
        print("Downloading raw MIMIC-IV Demo (v2.2) to data/raw/ ...")
        download_mimic_demo(out_dir=RAW_DIR, version="2.2", method="python")
    else:
        print("✓ Raw MIMIC-IV Demo already present.")

    out_paths = build_processed_extracts(hr_threshold=HR_THRESHOLD, t_max=T_MAX)
    print("✓ Built processed cohorts:")
    for k, v in out_paths.items():
        print(f"  {k}: {v}")
except Exception as e:
    print("Could not build processed cohort extracts in this environment.")
    print("Error:", e)
    print("If you already have the cohort file, place it in data/processed/ and re-run.")

### Optional: download raw MIMIC-IV Demo tables

Not required for the homework pipeline. Skip unless your instructor asks you to explore the raw Demo tables in `data/raw/`.

In [ ]:
# Optional: explore raw tables (safe to skip)
# The cohort-build cell above already downloads what you need for this lecture.

try:
    from data.download_data import download_mimic_demo

    download_mimic_demo(out_dir=RAW_DIR, version="2.2", method="python")
except Exception as e:
    print("Skipping raw MIMIC-IV Demo download.")
    print("Error:", e)

## Data
This lecture uses a **processed cohort extract** derived from MIMIC-IV Demo.

Expected file: `data/processed/cohort_L11_msm_longitudinal.parquet` (or `.csv`).

If this file is missing:
- Run the “Build the processed cohort extract” cell above, or
- Run locally: `python data/build_processed_extracts_demo.py --exposure-mode admission_type`

Assumed **long** format with columns like:
- `stay_id`, `t_day`
- treatment `A_t`
- time-varying covariates like `hr_mean`
- outcome `Y` at end

This notebook is **diagnostics-first**: you will compute stabilized weights using a provided scaffold, then **diagnose** and **interpret** the MSM estimate.


### Column definitions (this cohort)

These are defined by the cohort builder (`data/build_processed_extracts_demo.py`):

- **Unit + time index**: one row per ICU stay-day
  - `stay_id`: ICU stay identifier
  - `t_day`: day since ICU intime (0, 1, 2, ...)
- **Time-varying covariate**:
  - `hr_mean`: mean heart rate during that ICU day (from `chartevents`)
- **Time-varying treatment**:
  - `A_t = 1` if `hr_mean > HR_THRESHOLD` (a pedagogical stand-in for a time-varying intervention rule)
- **Outcome**:
  - `Y = 1` if the patient **died in-hospital** (`hospital_expire_flag`)


### Correction (local build command)

The `## Data` cell above may mention an older local build command.

For this lecture, the correct local build command is:

`python data/build_processed_extracts_demo.py --hr-threshold 100 --t-max 7`

(Or, in Colab, just run the build cell at the top of the notebook.)


### Before you load the cohort (important)

If you **do not** already have `data/processed/cohort_L11_msm_longitudinal.parquet` (or `.csv`):
- In Colab: run the cell titled **“Build the processed cohort extract (required)”** above, then come back here.
- Locally: run `python data/build_processed_extracts_demo.py --hr-threshold 100 --t-max 7`.


In [ ]:
# Statsmodels for regression (logit/ols); installed via requirements.txt
import statsmodels.api as sm
import statsmodels.formula.api as smf

parquet_path = PROC_DIR / "cohort_L11_msm_longitudinal.parquet"
csv_path = PROC_DIR / "cohort_L11_msm_longitudinal.csv"

if parquet_path.exists():
    df = pd.read_parquet(parquet_path)
elif csv_path.exists():
    df = pd.read_csv(csv_path)
else:
    raise FileNotFoundError(
        "Missing cohort file for L11. Build it via:\n"
        "  - Colab: run the build cell above\n"
        "  - Local: python data/build_processed_extracts_demo.py --hr-threshold 100 --t-max 7\n"
        "\nNote: This notebook focuses on treatment weights (W_A). Censoring weights (W_C) are an extension." 
    )

df.head()

### Variables


In [ ]:
# Columns in the built cohort (see data/build_processed_extracts_demo.py)
ID = "stay_id"
T = "t_day"
A = "A_t"  # time-varying treatment
Y = "Y"    # end-of-follow-up outcome column

# Create a simple numeric baseline covariate for modeling
if "sex_male" not in df.columns:
    df["sex_male"] = (df["sex"].astype(str).str.upper() == "M").astype(int)

# Time-varying covariates available in the extract
L_t = ["hr_mean"]

# Baseline covariates
L0 = ["age", "sex_male"]

cols_needed = [ID, T, A, Y] + L0 + L_t
missing = [c for c in cols_needed if c not in df.columns]
missing

## Part A — Stabilized treatment weights (scaffold)
You should NOT re-derive the formulas here. Focus on model specification and diagnostics.


In [ ]:
def stabilized_treatment_weights(data: pd.DataFrame) -> pd.DataFrame:
    d = data.copy()
    d = d.sort_values([ID, T]).reset_index(drop=True)

    # Denominator model: Pr(A_t=1 | past A, past L, baseline L0)
    # Numerator model:   Pr(A_t=1 | past A, baseline L0)
    # NOTE: The exact covariate history depends on your extract. This is a scaffold.

    # Create simple lag features (example: lagged treatment and lagged confounder)
    d["A_lag1"] = d.groupby(ID)[A].shift(1).fillna(0)
    for l in L_t:
        d[f"{l}_lag1"] = d.groupby(ID)[l].shift(1)

    denom_covs = ["A_lag1"] + L0 + [f"{l}_lag1" for l in L_t]
    num_covs   = ["A_lag1"] + L0

    denom_formula = A + " ~ " + " + ".join([c for c in denom_covs if c in d.columns])
    num_formula   = A + " ~ " + " + ".join([c for c in num_covs if c in d.columns])

    denom = smf.logit(denom_formula, data=d).fit(disp=False)
    num = smf.logit(num_formula, data=d).fit(disp=False)

    p_denom = denom.predict(d)
    p_num = num.predict(d)

    # Avoid division by zero
    eps = 1e-6
    p_denom = np.clip(p_denom, eps, 1-eps)
    p_num = np.clip(p_num, eps, 1-eps)

    w_t = np.where(d[A]==1, p_num/p_denom, (1-p_num)/(1-p_denom))
    d["sw_treat_t"] = w_t
    d["sw_treat"] = d.groupby(ID)["sw_treat_t"].cumprod()
    return d

d = stabilized_treatment_weights(df)
d[[ID,T,A,"sw_treat_t","sw_treat"]].head()

### TODO A1 — Diagnose weights
Plot `sw_treat` and choose a truncation rule.


In [ ]:
plt.figure()
plt.hist(d["sw_treat"], bins=80)
plt.xlabel("stabilized treatment weight (cumulative)")
plt.ylabel("count")
plt.title("MSM weights before truncation")
plt.show()

lo, hi = d["sw_treat"].quantile([0.01,0.99]).to_list()
d["sw_treat_trunc"] = d["sw_treat"].clip(lo,hi)
(lo,hi)

## Part B — Fit an MSM (simple)
Example MSM: E[Y] = β0 + β1 * cumulative_treatment.

Your extract may define treatment history differently; adapt as needed.


In [ ]:
# Example: cumulative treatment exposure up to time t
d["A_cum"] = d.groupby(ID)[A].cumsum()

# Use the last time point per subject for end-of-follow-up Y
last = d.sort_values([ID,T]).groupby(ID).tail(1).copy()

# TODO: define the MSM formula
msm_formula = Y + " ~ A_cum"  # or Y ~ A_t (if outcome is time-specific)

msm = smf.ols(msm_formula, data=last).fit(weights=last["sw_treat_trunc"])
msm.summary().tables[0]

### TODO B1 — Compare to naïve (unweighted) estimate


In [ ]:
naive = smf.ols(msm_formula, data=last).fit()
{"beta1_weighted": float(msm.params.get("A_cum", np.nan)),
 "beta1_naive": float(naive.params.get("A_cum", np.nan))}

## Reflection
1) What does the MSM coefficient represent here (marginal vs conditional)?
2) Which part is harder: computing weights or interpreting the estimate?
